Name: __________________        Class: __________________     Date: __________________

<center>
    <img width="100%" src="https://raw.githubusercontent.com/astroDimitrios/AstronomyClass/master/DesignAssets/Ai/Banner3plainCut.png" alt='AP Logo'>
</center>

# Ring Dynamics

<div class="alert alert-block alert-warning"> This activity is not maintained or updated. The Rebound portion of this activity will not work on Windows based systems as it is not supported by Rebound.</div>

## AIM - Visualise the Roche limit, identify resonances, and explain shepherding
Difficulty: Hard

**Predict**

Have a think about these questions and make some predictions! Be sure to tell someone else what you predict.

1) What effects the shape of the rings?  
2) How are gaps in the rings made?  
3) Do the outer moons of Saturn have an effect on the rings?  

**Enter your predictions:**    
1)    
2)    
3)    

## Contents

* [Start](#Start)
* [Resonances](#Resonances)
* [Waves](#Waves)
* [Shepherd's Moons](#SM)
* [Challenges](#Challenges)
* [References](#References)

# Let's go: Visualising the Roche limit <a class="anchor" id="Start"></a>

In this notebook we'll first visualise the roche limit which we calculated in the activity Planetary Rings using an N-body simulation with the rebound package, then we will look at mean-motion resonances using data on the moons of Saturn, and finally we will briefly look at the effects of certain 'shepherd' moons.

An N-body simulation is a simulation of many particles under the influence of gravity and other physical laws. We will use the rebound package to simulate what happens to a loosely held together mass of particles as it approaches Saturn and breaks up! The following code block sets up our simulation:

In [ ]:
# run me
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib.patches import Circle
import rebound
from IPython.display import Image

In [ ]:
# don't change anything just run it!
sim = rebound.Simulation()        # set's up the simulation
sim.units = ('km', 's', 'kg')     # set's the units for the sim
sim.dt = 10                       # set's the time step - this is in s which we defined above
sim.softening = 0.2               # smoothes out numerical artefacts at small scales
sim.gravity    = "basic"          # set's the gravity module

Let's start by adding Saturn. This code queries NASA's Horizons database for the current location of Saturn. Then we move to the new centre of mass of the system (Saturn) and print the simulation status:

In [ ]:
# don't change anything just run it!
sim.add("699")       # add Saturn using its code 699
sim.move_to_com()    # move to the new centre of mass
sim.status()         # print the status of the sim

The ```sim.status()``` call shows the current timestep and the current time of the simulation along with the position of the particles in our simulation. 

Now we need to add our loosely bound mass of particles which will represent our moon which will get too close to Saturn and break up. Complete the code block below to add the particles:

In [ ]:
# we will pick random particle radii based on a powerlaw
def powerlaw(slope, min_v, max_v):
    y = np.random.uniform()
    pow_max = pow(max_v, slope+1.)
    pow_min = pow(min_v, slope+1.)
    return pow((pow_max-pow_min)*y + pow_min, 1./(slope+1.))

particle_density = .1 # kg/m^3

print('Adding ring particles')
count = 0
# this while loop will add 100 particles to make our moon
while count < 100:
    radius = powerlaw(slope=-4, min_v=1, max_v=4)/1000  # m get the radius
    # calculate the mass from the radius and the particle_density
    # assume the particle is a spere
    # Your code here:
    mass = '''Your code here'''  # kg
    rs = 180000 # our particles will be 180,000 km away from the centre of Saturn
    theta = np.pi/4
    # convert from polar coords to cartesian
    x = rs*np.cos(theta)
    y = rs*np.sin(theta)
    # use np.random.uniform() to add a number between -5000 and +5000 to the x and y coords
    # this will spread our particles out so they're not all on top of each other
    # hint np.random.uniform() has a 'low' and 'high' keyword argument
    # Your code here:
    x += '''Your code here'''
    y += '''Your code here'''
    # lastly use v = sqrt(GM/r) to calculate the speed of the particle
    # G = 6.67428e-11, M is the mass of Saturn (stored in sim.particles[0].m)
    # r is the distance rs but in m not km, convert the final velocity into km/s and times by .75
    # this will ensure our moon is falling towards Saturn
    # Your code here:
    v = '''Your code here''' # km/s
    # this sim.add() call adds this new particle!
    sim.add(
        m=mass,
        r=radius,
        x=x,
        y=y,
        z=np.random.normal(),
        vx = -v*np.sin(theta),
        vy = v*np.cos(theta),
        vz = 0.)
    count += 1
print('Finished adding ring particles')

Now our simulation is set up with all our particles! You can use ```sim.status()``` again now if you like to see the particle list (this will print out a lot!). Now we need to visualise the system after each integration or jump forward in time. To do this I have defined a plotting function below which will generate images in the myimages folder which you can download at the end of the activity. Run the code block now then move on:

In [ ]:
# don't change anything just run it!
def plotParticles(sim, k):
    '''
    Takes an input simulation and integration number k
    Outputs a png of the current system with Saturn scaled to real size but particles enlarged
    '''
    fig = plt.figure(figsize=(8,8))
    ax = plt.subplot(111, aspect='equal')
    ax.set_ylabel("y / km")
    ax.set_xlabel("x / km")
    ax.set_xlim(-200000, 200000)
    ax.set_ylim(-200000, 200000)
    ax.set_aspect('equal')
    ax.ticklabel_format(axis="both", style="sci", scilimits=(0,0))
    for i, p in enumerate(sim.particles):
        if i == 0:
            fc, ec, a, r = "goldenrod", "None", 1, 58232
        else:
            fc, ec, a, r = "lightgray", "k", 1, p.r*5000
        circ = Circle((p.x, p.y), r, facecolor=fc, edgecolor=ec, alpha=a)
        ax.add_patch(circ)
    plt.savefig('./myimages/dynamics_'+str(k)+'.png', dpi=100)
    fig.clf()
    plt.close()

Now we can run our simulation! The next code block will integrate forward 100 times (jumping 100 s each time) and output an image after each jump:

In [ ]:
# This should only take ~ 30s
plotParticles(sim, 0)                 # plot the initial setup!
for i in range(100):                  # jump 100 times
    sim.integrate(sim.t + 100)        # this gets the current sime time (sim.t) and jumps an extra 100 s
    plotParticles(sim, i+1)           # plot again

sim.save("./myimages/myfirstsim.bin")               # this will save the last state of the particles in the sim

Now check in the myimages folder and you should see 101 images with the prefix 'dynamics'! Download them and make a gif from them (perhaps using [ezgif](https://ezgif.com/)). You should end up with something like this:

In [ ]:
Image('Images/roche.gif')

Our moon disintegrates! Remember the side of the moon closest to Saturn experiences a stronger gravitational pull and also has to travel faster to maintain its orbit so the particles get spread out in a band. Now you know how to create a basic N-body simulation you can adapt it and make it your own (maybe tackle challenge 3!).

# Resonances <a class="anchor" id="Resonances"></a>

A resonance occurs when the orbital period of one particle is an integer multiple of another particles orbital period. Take the 2:1 resonance for example. This means the inner particle (the one closer to the parent body) orbits twice in the time the outer particle orbits only once. This is an example of a first order resonance. The first order resonances are 2:1, 3:2, 4:3, 5:4, 6:5 and so on! 

When two objects are in resonance they 'tug' on each other at certain points around their orbits. These small gravitational tugs build up to a point where they can significantly change each others orbits. Resonances can either stabilise the orbits of the objects (making it harder for them to change their orbit) or destabilise the orbits of one or both of the objects.

In this section we'll first look at the moons to see if any are in resonant orbits with each other. Then we'll determine locations in the rings where the ring particles are resonant with certain moons and link these resonances to ring features.

Let's start by loading the csv containing data on Saturn's moons:

In [ ]:
saturn_moons = pd.read_csv('saturn_moons.csv')
saturn_moons.head()

For now we are only really interested in the moon's Name and Period. To calculate whether two moons are in resonant orbits with each other we will take one moons period and divide it by another moons period. If we get any ratio of integers there is a resonance! For instance if after dividing we get 1.5 (or close) the two moons are in a 3:2 resonance! Complete the code block below to create a new dataframe with the moon names as both the row and column names and the ratio of periods as the data values:

In [ ]:
# get the names of the moons from the dataframe
# Your code here:
moon_names = '''Your code here'''
# This will create an empty df for you!
moon_res = pd.DataFrame({}, columns=moon_names)

# loop over each name
for i in moon_names:
    # get the period for this moon, i, from the first df
    # Your code here:
    this_moon_period = '''Your code here'''
    res = []
    # loop over moon names again
    for j in moon_names:
        # get the period for the moon with name j from the first df and divide it by this_moon_period
        # append it to res
        # Your code here:
        period_ratio = '''Your code here'''
        res.append(period_ratio)
    # finally these two lines add this new row to our empty df which we called moon_res
    this_row = pd.DataFrame([res], index=[i], columns=moon_names)
    moon_res = pd.concat([moon_res, this_row])

moon_res

Now we have a table with period ratios! You need only look at the numbers above the diagonal (which are all ones). Which moons are in resonant orbits with each other? Remember the number will need to be close to the ratio of two integers.

Let's now look at resonances between the moons and ring particles. Specifically those with the moon Mimas (the death star moon). We need to get Mimas' Period and Semimajor Axis from our first dataframe. Divide Mimas' period by an integer (2 for the 2:1 resonance) and then use Kepler's 3rd law:

$$A^3 = kT^3 $$

to find the Semimajor Axis ($A$) of the resonant ring particle with period $T$. Here $k$ is a constant that we will have to calculate. Complete the code block below to calculate the location of the 2:1 resonance with Mimas:

In [ ]:
# locate the radius and period of Mimas from the dataframe saturn_moons
# Your code here:
mimas_radius = saturn_moons.loc[saturn_moons['Name'] == 'Mimas', 'Semimajor Axis'].values[0]
mimas_period = '''Your code here'''
print('Mimas a = {:6.0f} km'.format(mimas_radius))
print('Mimas T = {:6.2f} days'.format(mimas_period))

# calculate the constant using Kepler's 3rd law (re-arrange to get k)
# Your code here:
const = '''Your code here''' # km^3 / days^2

# divide the period by 2 to get mimas_2_1_period
# then use Kepler's 3rd law to get A (mimas_2_1_radius)
# Your code here:
mimas_2_1_period = '''Your code here'''
mimas_2_1_radius = '''Your code here'''
print('Mimas 2:1 res a = {:6.0f} km'.format(mimas_2_1_radius))
print('Mimas 2:1 res T = {:6.2f} days'.format(mimas_2_1_period))

Use the reference image below to identify the feature which lies near this resonance:

In [ ]:
Image('Images/Ring_overview.jpg')

Now use the code block below to calculate the resonances with the large moon Janus and then identify ring features at those resonant positions (if there are any):

In [ ]:
def locate_res(moon_name, order):
    '''
    Takes the name of a moon of saturn as a string and returns the resonant periods and locations
    '''
    # get the radius and period of the moon with name moon_name
    # Your code here:
    moon_radius = '''Your code here'''
    moon_period = '''Your code here'''
    print(moon_name+' a = {:6.0f} km'.format(moon_radius))
    print(moon_name+' T = {:6.2f} days'.format(moon_period))
    # calculate the constant k
    # Your code here:
    this_const = '''Your code here''' # km^3 / days^2
    res_no = [2, 3, 4, 5, 6, 7]
    # loop over each number in res_no
    # calculate the period and radius of the particle
    # Hint: divide the moon_period by the number in res_no then times by the number minus the order
    # this will let us calculate second and higher order resonances
    # Your code here:
    for i in res_no:
        this_res_period = '''Your code here'''
        this_res_radius = '''Your code here'''
        print(moon_name+' '+str(i)+':'+str(i-order)+' a = {:6.0f} km'.format(this_res_radius))
        print(moon_name+' '+str(i)+':'+str(i-order)+' T = {:6.2f} days'.format(this_res_period))

# This will print the first order resonances with the moon Janus!
locate_res('Janus', 1)

Load the image below and use it and the one above to locate ring features at the resonant positions with Janus.

In [ ]:
Image('Images/Aring_overview.jpg')

## Waves <a class="anchor" id="Waves"></a>

The next image was located using the [OPUS](http://pds-rings.seti.org/search/ ) search tool on the rings and moons node of the planetary data system. It covers a range of 131,500 km to 132,500 km in semimajor axis and was taken by the Cassini spacecraft!

In [ ]:
Image('Images/N1467351539_2.IMG.jpeg')

The inner wave on the far left is a bending wave. Bending waves move inwards (towards Saturn) and are produced by moons which are inclined relative to Saturn's equator and the plane of Saturn's rings. Bending waves create vertical ripples in the ring. The larger wave to the right of the bending wave (just left of centre at the bottom) is a density wave which moves outwards. Density waves are more common than bending waves. To the right of the image you can see another pair of bending and density waves!

One of the larger inner moons causes the bending and density waves between 131,500 km and 132,500 km. Can you identify which one it is? Remember to create a bending wave the moon must have a non-zero inclination (i). Use the code block below and the function we wrote above to calculate the resonances for your guess of moon:

In [ ]:
# Hint the resonance you're looking for is NOT first order!
# Your code here:
locate_res('Mimas', 2)

## Shepherd Moons <a class="anchor" id="SM"></a>

Finally let's briefly look at shepherd moons. These are moons that lie in gaps in Saturn's rings. Their gravitational pull on the ring particles clears out the area around them. In the following image you can see the moon Daphnis in the Keeler gap:

In [ ]:
Image('Images/daphnis1.jpg')

The moons gravity clears out the material from the gap. Notice how it is also creating ripples in the edges of the gap. This pattern forms from the combined attraction from the moon and Saturn. The ripple along the inner edge (at the top) are leading Daphnis ie they are ahead of Daphnis in their orbit because they are travelling at a higher speed. The ripple along the outer edge trails Daphnis. These ripples are prominent because the gap is quite small/Daphnis is close to the edges. The Encke gap in which the moon Pan resides does not have ripples like this since the gap is much wider and there is a larger distance inbetween Pan and the gap edges.

You can also use rebound to simulate the formation of these ripples! The gif you can load below shows my attempt at simulating a small gap and the effect of a moon like Daphnis on a ring arc (a segment of a ring):

In [ ]:
Image('Images/shepherd.gif')

There are also other ring features such as spokes which I haven't mentioned! You should check out the references below for more on Saturn's rings.

## Challenges: <a class="anchor" id="Challenges"></a>

1) Identify features related to resonances with Enceladus. Are there any?  
2) Use the OPUS tool to find images to make a gif of the ripples or a moon moving!  
3) Use rebound or another N-body simulation package to simulate a shepherd moon.  
4) Follow the two ring simulation articles in the references below to create your own ring simulation from first principles.  

# References <a class="anchor" id="References"></a>

Phil Nicholson - Resonances and Rings [http://hosting.astro.cornell.edu/specialprograms/reu2012/workshops/rings/](http://hosting.astro.cornell.edu/specialprograms/reu2012/workshops/rings/)  
Outer Planets Data Search Tool (OPUS3) PDS Ring-Moons Node NASA/SETI [https://opus.pds-rings.seti.org/opus/](https://opus.pds-rings.seti.org/opus/)  
Rebound N-body simulator [https://rebound.readthedocs.io/en/latest/](https://rebound.readthedocs.io/en/latest/)  

Chris Mihos - Ring Dynamics [http://burro.astr.cwru.edu/Academics/Astr221/SolarSys/Rings/dynamics.html](http://burro.astr.cwru.edu/Academics/Astr221/SolarSys/Rings/dynamics.html)  
Matthew Hedman - Planetary Ring Dynamics [https://webpages.uidaho.edu/mhedman/papers_published/UNESCO_dynamics.pdf](https://webpages.uidaho.edu/mhedman/papers_published/UNESCO_dynamics.pdf)     
Shane Byrne - Rings and Moons of Saturn [http://www.lpl.arizona.edu/~shane/PTYS_206/lectures/PTYS_206_saturns_rings_moons.pdf](http://www.lpl.arizona.edu/~shane/PTYS_206/lectures/PTYS_206_saturns_rings_moons.pdf)  

Simulations:  
Cole Kendrick - Computer Simulation of Saturn's Rings [http://courses.physics.ucsd.edu/2018/Winter/physics141/Assignments/SaturnRingSimulation.pdf](http://courses.physics.ucsd.edu/2018/Winter/physics141/Assignments/SaturnRingSimulation.pdf)  
Kirsten Larson - The Effects of Moons on Saturn's Ring System [http://physics.wooster.edu/JrIS/Files/Larson_Web_article.pdf](http://physics.wooster.edu/JrIS/Files/Larson_Web_article.pdf)

## Sharing

If you share, use or modify this activity in any way use the citation in this [txt file](https://github.com/astroDimitrios/Astronomy/blob/master/CITATION.txt).    
Please contact me at astrodimitrios@gmail.com with any suggestions, mistakes found, or general questions about teaching astronomy with Python.

© Dimitrios Theodorakis GNU General Public License v3.0 https://github.com/astroDimitrios/Astronomy 